# RTX 远程压测（从本机打到 RTX）

这个 Notebook 用于在“本机/压测机”远程压测 RTX 上运行的 Proxy，并生成 `project.txt` 里常见的指标：

- **10K 并发连接**（`connect_hold`）
- **HTTP /stats 延迟与 QPS**（`http_stats`）

依赖：仓库自带脚本 `scripts/benchmark.py` / `scripts/load_test.py`。

提示：10K 连接需要较高 fd 上限；请在启动 Jupyter 前执行：

```bash
ulimit -n 200000
```


In [ ]:
import os, sys, json, time, subprocess, textwrap
from pathlib import Path
import resource

# 目标 RTX（按需修改）
RTX_HOST = "10.200.98.98"

# RTX 上 proxy 的端口（常规部署）
PROXY_PORT = 8080

# 如果你在 RTX 上跑 ai_demo（端口默认 18080），可用于“巨大计算需求”演示
AI_DEMO_PORT = 18080

# 输出目录
OUT_DIR = Path("bench_out")
OUT_DIR.mkdir(parents=True, exist_ok=True)

print("OUT_DIR=", OUT_DIR.resolve())
print("Python=", sys.version.split()[0])
print("Repo=", Path('.').resolve())

nofile_soft, nofile_hard = resource.getrlimit(resource.RLIMIT_NOFILE)
print(f"RLIMIT_NOFILE soft={nofile_soft} hard={nofile_hard}")
if nofile_soft < 20000:
    print("WARNING: 当前进程 fd 上限偏低；10K 并发连接可能失败。建议退出后用更高 ulimit 启动 Jupyter。")


## 1) 快速探活：拉取 `/stats`

确认本机到 RTX 网络可达、并且 proxy 正在运行。

In [ ]:
import urllib.request

def get_json(url: str, timeout: float = 2.0):
    with urllib.request.urlopen(url, timeout=timeout) as r:
        data = r.read()
    return json.loads(data.decode('utf-8'))

stats_url = f"http://{RTX_HOST}:{PROXY_PORT}/stats"
j = get_json(stats_url, timeout=3.0)
print("OK /stats")
print("uptime_sec:", j.get("uptime_sec"))
print("active_connections:", j.get("active_connections"))
print("avg_qps:", j.get("avg_qps"))
print("io:", j.get("io"))


## 2) 10K 并发连接压测（connect_hold）

这个测试从本机建立大量 TCP 连接到 RTX 的 proxy，并保持一段时间。

参数建议：
- `--total 10000`：目标连接数
- `--concurrency 2000`：建立连接的并发度（过大可能让本机端口/FD/网络瞬间吃满）
- `--hold 20`：保持时间（秒）

同时你可以打开 RTX 上的仪表盘看 `active_connections`：
- `http://10.200.98.98:8080/dashboard`


In [ ]:
def run_cmd(cmd: list[str], timeout_s: float | None = None) -> str:
    print("$", " ".join(cmd))
    p = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, timeout=timeout_s)
    print(p.stdout)
    p.check_returncode()
    return p.stdout

out_json = OUT_DIR / "remote_conn_hold_10k.json"
cmd = [
    sys.executable, "scripts/benchmark.py",
    "--host", RTX_HOST,
    "--port", str(PROXY_PORT),
    "--bench", "connect_hold",
    "--total", "10000",
    "--concurrency", "2000",
    "--hold", "20",
    "--timeout", "3",
    "--global-timeout", "90",
    "--output", str(out_json),
]
run_cmd(cmd, timeout_s=120)
print("saved:", out_json)


In [ ]:
data = json.loads(out_json.read_text(encoding='utf-8'))
print(json.dumps(data, ensure_ascii=False, indent=2)[:2000])


## 3) /stats 延迟与吞吐压测（http_stats）

这个测试会高并发请求 RTX 的 `GET /stats`，输出 p50/p90/p99 延迟和 QPS。

注意：`/stats` 不是数据面业务请求，但适合做“可观测接口在高频拉取下是否稳定”的基准。

In [ ]:
out_json = OUT_DIR / "remote_http_stats.json"
cmd = [
    sys.executable, "scripts/benchmark.py",
    "--host", RTX_HOST,
    "--port", str(PROXY_PORT),
    "--bench", "http_stats",
    "--total", "20000",
    "--concurrency", "400",
    "--timeout", "2",
    "--global-timeout", "120",
    "--output", str(out_json),
]
run_cmd(cmd, timeout_s=150)
print("saved:", out_json)


In [ ]:
data = json.loads(out_json.read_text(encoding='utf-8'))
print(json.dumps(data, ensure_ascii=False, indent=2)[:2000])

# 可选画图：延迟分布（如果你的环境有 matplotlib）
try:
    import matplotlib.pyplot as plt
    lat = data.get('latency_ms', {})
    if lat:
        xs = ['p50','p90','p99','avg']
        ys = [lat.get('p50_ms',0), lat.get('p90_ms',0), lat.get('p99_ms',0), lat.get('avg_ms',0)]
        plt.figure(figsize=(6,3))
        plt.bar(xs, ys)
        plt.title('GET /stats latency (ms)')
        plt.ylabel('ms')
        plt.show()
except Exception as e:
    print('matplotlib unavailable:', e)


## 4) （可选）“巨大计算需求”压测到 RTX demo

如果你在 RTX 上启动了 `ai_demo`（端口 `18080`，路径 `/infer?work_ms=...`），就用下面脚本从本机远程压测：

```bash
python3 scripts/load_test.py --base http://10.200.98.98:18080 --path /infer --duration 30 --concurrency 400 --work-ms 800 --mode spread
```

运行后打开：
- `http://10.200.98.98:18080/dashboard`

你会在“后端 AI/GPU 指标表格 + GPU/显存/队列曲线”里看到分配与负载变化。